In [ ]:
%pip install openai
%pip install PyPDF2
%pip install openpyxl  


In [9]:
import json
import os  

from openai import AzureOpenAI
from PyPDF2 import PdfReader
from openpyxl import Workbook  

In [ ]:
COMPLETIONS_MODEL = "GPT4-1106"

client = AzureOpenAI(
    azure_endpoint="https://nuevayorkopenaiservice.openai.azure.com/",
    api_key="77c21ef531224c41b2581310f11e4aea",
    api_version="2023-12-01-preview",
)

In [ ]:
class Opportunity:  
    def __init__(self, title, content):  
        self.title = title  
        self.content = content  
  
def read_pdf_content(pdf_path): 
    reader = PdfReader(pdf_path) 
    text = ""  
    for page in reader.pages:  
        text += page.extract_text()
    return text  
  
def get_opportunities_from_directory(directory_path):  
    opportunities = []  
      
    for filename in os.listdir(directory_path):  
        if filename.endswith('.pdf'):  
            file_path = os.path.join(directory_path, filename)  
            content = read_pdf_content(file_path)  
            title = os.path.splitext(filename)[0]  
            opportunity = Opportunity(title, content)  
            opportunities.append(opportunity)  
      
    return opportunities  
  
# Uso de la función en un directorio específico  
directory_path = 'pdfs'  # Reemplaza con la ruta de tu directorio  
opportunities = get_opportunities_from_directory(directory_path)  
  
# Ahora tienes una lista de objetos Opportunity con títulos y contenidos  
for opp in opportunities:  
    print(f'Título: {opp.title}\nContenido: {opp.content[:100]}...')  # Muestra los primeros 100 caracteres  

In [ ]:
def run_conversation():
    # Step 1: send the conversation and available functions to the model
    messages = [
        {"role": "system", "content": "Enagás es una empresa del sector energético"},
        {"role": "user", "content":"¿Cuál es el sector de Enagás?"}
    ]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_evaluation",
                "description": "Get the evaluation",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "evaluation_score": {
                            "type": "integer",
                            "description": "Score",
                        },
                        "reasoning": {
                            "type": "string",
                            "description": "A reasoning of how the answer was created"},
                        },
                    "required": ["evaluation_score"],
                },
            },
        }
    ]

    response = client.chat.completions.create(
        model=COMPLETIONS_MODEL,
        messages=messages,
        tools=tools,
        tool_choice={"type": "function", "function": {"name": "get_evaluation"}},
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls

    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        json_str = tool_calls[0].function.arguments
        dict_args = json.loads(json_str)
        evaluation_score = dict_args['evaluation_score']
        reasoning = dict_args['reasoning']
        print("Evaluation score: " + str(evaluation_score))
        print("Reasoning: " + reasoning)

    return response_message

In [ ]:
print(run_conversation())

## Guardar resultados en xlsx

In [10]:
def save_opportunities_to_excel(opportunities, excel_path):  
    # Crear un nuevo libro de trabajo de Excel  
    wb = Workbook()  
    ws = wb.active  
      
    # Agregar los encabezados de las columnas  
    ws.append(['Título', 'Contenido'])  
      
    # Agregar las oportunidades al libro de trabajo  
    for opp in opportunities:  
        ws.append([opp.title, opp.content])  
      
    # Guardar el libro de trabajo en un archivo  
    wb.save(excel_path)  
  
# Suponiendo que `opportunities` es la lista de objetos Opportunity que obtuviste anteriormente  
excel_path = 'resultados.xlsx'  # Reemplaza con la ruta de tu archivo deseado  
save_opportunities_to_excel(opportunities, excel_path)